# HW2 part 2: GNN 

## Graph Attention Network (GAT) Implementation for Node Classification

**Objective:** Implement a GAT from scratch to perform node classification using an OGB dataset. Develop neural components, including the forward pass, as well as training and testing routines. There are 17 `todo`s and 2 questions for GAT.

A Graph Attention Network (GAT) applies the concept of self-attention to graph-structured data. Unlike traditional Graph Convolutional Networks (GCNs) that rely on fixed or uniform weights derived from adjacency structures, GAT learns to assign different weights (attention scores) to different edges. This allows the model to focus more on important neighbors while possibly ignoring less relevant ones. By doing so, GAT effectively captures how each node interacts with its neighbors in a more flexible and adaptive way.

In [1]:
# This notebook's first part demonstrates how to train and evaluate a GAT model on the ogbn-arxiv dataset for node classification (To predict the category of each paper). Make sure you know the basics of GNNs, PyG and pytorch before starting this notebook. If not, please check the corresponding tutorials first.

# If you use Google Colab, you can uncomment and run the following command to install the required packages.
# For this assigment, we recommend using your own local computer since the RAM usage is high, Colab may crash due to the high RAM usage.
# We passed our assignment on our local computers using python version 3.10.16.
# !pip install torch==2.5.0
# !pip install torch-geometric==2.6.1
# !pip install ogb==1.3.6

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GATConv
from ogb.nodeproppred import PygNodePropPredDataset,Evaluator

# We recommend to use GPU for training if possible. Because GAT is a relatively large model, training on CPU can be slow. But after testing, we find that training GAT on CPU is also acceptable.


# if torch.cuda.is_available():
#     device = torch.device("cuda")  # NVIDIA GPU
# else:
#     device = torch.device("cpu")   # CPU fallback
device = torch.device("cpu")   # CPU fallback
print(f"Using device: {device}")


import random
def set_seed(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    # make cudnn deterministic
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# set a random seed for reproducibility
set_seed(42)

Using device: cpu


In [2]:
# ## Define a GAT Model for Node Classification

#### Model Structure

## The model is divided into two GATConv layers:
##   The first layer uses multi-head attention (specified by num_heads) to produce richer representations. Here, each head outputs hidden_channels features, and they are concatenated, resulting in a total output dimension of hidden_channels * num_heads.
##   The second layer is a single-head output layer that directly predicts the final node embeddings or categories. Its output dimension corresponds to the number of classes (out_channels).

#### Layer Normalization

## After the first GAT layer, the output is passed through nn.LayerNorm(hidden_channels * num_heads). This helps stabilize training by normalizing feature distributions across different nodes.

#### Forward Pass

## First apply dropout to the input features (x) to reduce overfitting.
## Pass the data through the first GAT layer (gat1).
## Apply an ELU activation, then layer normalization, and another dropout.
## Finally, pass the features through the second GAT layer (gat2) to get the predictions (the paper category with the highest score).

class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_heads=8, dropout=0.6,add_self_loops=True):
        super(GAT, self).__init__()
        self.dropout = dropout
        # The first layer: multi-head attention, each head outputs hidden_channels, and the total output dimension = hidden_channels * num_heads (i.e. the input dimension of the second layer), you don't need to care the concatenation in this layer
        # The usage of GATConv is: GATConv(in_channels, out_channels, heads=XXX, concat=XXX, dropout=dropout, add_self_loops=XXX)
        ## TODO 1: Define the first GAT layer (2 points)

        self.gat1 = GATConv(in_channels, hidden_channels, heads=num_heads, concat=True, dropout=dropout, add_self_loops=add_self_loops)
        
        ## TODO 1: Define the first GAT layer

        # The second layer: single-head output, not concatenated, directly output the dimension of the node categories
        ## TODO 2: Define the second GAT layer (2 points)

        self.gat2 = GATConv(hidden_channels * num_heads, out_channels, heads=1, concat=False, dropout=dropout, add_self_loops=add_self_loops)
        
        ## TODO 2: Define the second GAT layer

        # Layer normalization for stable training, use nn.LayerNorm, the input is the hidden_channels * num_heads
        ## TODO 3: Define a LayerNorm layer (2 points)
        
        self.layer_norm = nn.LayerNorm(hidden_channels * num_heads)

        ## TODO 3: Define a LayerNorm layer

    def forward(self, data):
        # data contains x and edge_index
        x, edge_index = data.x, data.edge_index

        ## TODO 4: use a dropout layer for the input features, then apply the first GAT layer (2 points)
        # use F.dropout to perform dropout, the input is x, and the dropout rate is self.dropout, set training=self.training

        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.gat1(x, edge_index)

        ## TODO 4: use a dropout layer for the input features, then apply the first GAT layer

        ## TODO 5: apply ELU activation, layer normalization and dropout (2 points)
        # use F.elu for ELU activation, the input is x

        x = F.elu(x)
        x = self.layer_norm(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        ## TODO 5: apply ELU activation, layer normalization and dropout

        ## TODO 6: apply the second GAT layer (2 points)

        x = self.gat2(x, edge_index)

        ## TODO 6: apply the second GAT layer
        return x

# Load the ogbn-arxiv dataset
dataset_arxiv = PygNodePropPredDataset(name='ogbn-arxiv')

# Let's see some properties of the dataset
## TODO 7: print the number of graphs in the dataset (2 points)

print(f"Number of graphs in dataset: {len(dataset_arxiv)}")

## TODO 7: print the number of graphs in the dataset

# Let's see how many features and classes are in the dataset
## TODO 8: print the number of features and classes in the dataset (2 points)

print(f"Number of features: {dataset_arxiv.num_features}, Number of classes: {dataset_arxiv.num_classes}")

## TODO 8: print the number of features and classes in the dataset


data_arxiv = dataset_arxiv[0]
# Let's see the shape of the node features and the target labels
print(data_arxiv.x.shape)

# Get the data split index
split_idx_arxiv = dataset_arxiv.get_idx_split()
train_idx_arxiv = split_idx_arxiv['train']
valid_idx_arxiv = split_idx_arxiv['valid']
test_idx_arxiv  = split_idx_arxiv['test']

# Model, optimizer, scheduler, and evaluator settings
model_arxiv = GAT(in_channels=dataset_arxiv.num_features,
                         hidden_channels=64,
                         out_channels=dataset_arxiv.num_classes,
                         num_heads=8,
                         dropout=0.6).to(device)
data_arxiv = data_arxiv.to(device)
optimizer_arxiv = torch.optim.Adam(model_arxiv.parameters(), lr=0.005, weight_decay=5e-4)
scheduler_arxiv = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_arxiv, mode='min', factor=0.7, patience=10, verbose=True)
evaluator_arxiv = Evaluator(name='ogbn-arxiv')

def train_arxiv():
    model_arxiv.train()
    optimizer_arxiv.zero_grad()
    ## TODO 9: forward propagation, loss calculation and backward propagation (2 points)
    # use F.cross_entropy as the loss function, the input is the output of the model and the target labels, and only use the training set for loss calculation, then call loss.backward()
    
    out = model_arxiv(data_arxiv)
    loss = F.cross_entropy(out[train_idx_arxiv], data_arxiv.y[train_idx_arxiv].squeeze())
    loss.backward()

    ## TODO 9: forward propagation, loss calculation and backward propagation
    optimizer_arxiv.step()
    return loss.item()

@torch.no_grad()
def evaluate_arxiv():
    model_arxiv.eval()
    ## TODO 10: get the prediction and use argmax to get the final category (2 points)

    out = model_arxiv(data_arxiv)
    y_pred = out.argmax(dim=1).unsqueeze(1)

    ## TODO 10: get the prediction and use argmax to get the final category
    train_acc = evaluator_arxiv.eval({'y_true': data_arxiv.y[train_idx_arxiv],
                                      'y_pred': y_pred[train_idx_arxiv]})['acc']
    valid_acc = evaluator_arxiv.eval({'y_true': data_arxiv.y[valid_idx_arxiv],
                                      'y_pred': y_pred[valid_idx_arxiv]})['acc']
    test_acc  = evaluator_arxiv.eval({'y_true': data_arxiv.y[test_idx_arxiv],
                                      'y_pred': y_pred[test_idx_arxiv]})['acc']
    return train_acc, valid_acc, test_acc

Number of graphs in dataset: 1
Number of features: 128, Number of classes: 40
torch.Size([169343, 128])


c:\Users\monat\AppData\Local\Programs\Python\Python311\Lib\site-packages\ogb\nodeproppred\dataset_pyg.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.sl

In [3]:
## You can change these hyperparameters to see if you can get better results, but the default hyperparameters should work. And also make sure the three sets for hyperparameters are the same.
num_epochs = 30
best_valid_acc_arxiv = 0
patience_arxiv = 30
trigger_times_arxiv = 0
best_model_state_arxiv = None

In [ ]:
# import os
# import gc

# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
# torch.cuda.empty_cache()
# gc.collect()

In [4]:
print("---- ogbn-arxiv training start ----")
for epoch in range(1, num_epochs + 1):
    loss = train_arxiv()
    scheduler_arxiv.step(loss)
    train_acc, valid_acc, test_acc = evaluate_arxiv()
    ## TODO 11: early stopping, in an if-else block (2 points)
    
    if valid_acc > best_valid_acc_arxiv:
        best_valid_acc_arxiv = valid_acc
        best_model_state_arxiv = model_arxiv.state_dict()
        trigger_times_arxiv = 0  # Reset trigger count when improvement is observed
    else:
        trigger_times_arxiv += 1  # Increment trigger count when no improvement
       
    ## TODO 11: early stopping, in an if-else block

    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, Valid: {valid_acc:.4f}, Test: {test_acc:.4f}')

    if trigger_times_arxiv >= patience_arxiv:
        print("Early stopping triggered!")
        break

---- ogbn-arxiv training start ----
Epoch: 001, Loss: 6.1625, Train: 0.1791, Valid: 0.0764, Test: 0.0588
Epoch: 002, Loss: 5.5791, Train: 0.1312, Valid: 0.2622, Test: 0.2769
Epoch: 003, Loss: 5.3758, Train: 0.1888, Valid: 0.2703, Test: 0.2310
Epoch: 004, Loss: 5.0632, Train: 0.2032, Valid: 0.2394, Test: 0.2043
Epoch: 005, Loss: 4.7448, Train: 0.3108, Valid: 0.2866, Test: 0.2442
Epoch: 006, Loss: 4.3284, Train: 0.2624, Valid: 0.2499, Test: 0.2101
Epoch: 007, Loss: 4.2126, Train: 0.2613, Valid: 0.2461, Test: 0.2066
Epoch: 008, Loss: 4.2357, Train: 0.2906, Valid: 0.2955, Test: 0.2542
Epoch: 009, Loss: 4.0538, Train: 0.3509, Valid: 0.3923, Test: 0.3838
Epoch: 010, Loss: 3.8687, Train: 0.2964, Valid: 0.3573, Test: 0.3918
Epoch: 011, Loss: 3.7791, Train: 0.3802, Valid: 0.4532, Test: 0.4518
Epoch: 012, Loss: 3.6805, Train: 0.4049, Valid: 0.4212, Test: 0.3706
Epoch: 013, Loss: 3.6393, Train: 0.4142, Valid: 0.4163, Test: 0.3546
Epoch: 014, Loss: 3.5714, Train: 0.4137, Valid: 0.4062, Test: 0.350

In [5]:
# Load the best model state
model_arxiv.load_state_dict(best_model_state_arxiv)
final_train, final_valid, final_test = evaluate_arxiv()
print(f"[ogbn-arxiv] Best validation accuracy: {final_valid:.4f}, corresponding test accuracy: {final_test:.4f}")

[ogbn-arxiv] Best validation accuracy: 0.5099, corresponding test accuracy: 0.4793


* What's the best validation accuracy you can get? (2 points) What's the corresponding test accuracy? (2 points) Please report the results in this markdown cell.
<br><br>**ANS:**
Best Validation Accuracy: 0.5099
Corresponding Test Accuracy: 0.4793
These are the best results from the ogbn-arxiv dataset after training Graph Attention Network (GAT) model.

* Let's see if we remove the graph structure and only use the node features, how well the model can perform.

In [7]:
import copy

# construct a dummy edge_index with self-loops only
num_nodes = data_arxiv.num_nodes
dummy_edge_index = torch.arange(num_nodes, device=data_arxiv.x.device).unsqueeze(0).repeat(2, 1)

# copy the original data and replace the edge_index with the dummy one
data_arxiv_no_graph = copy.deepcopy(data_arxiv)
data_arxiv_no_graph.edge_index = dummy_edge_index

# define a new model for the data without graph structure, with the same hyperparameters
model_no_graph = GAT(
    in_channels=dataset_arxiv.num_features,
    hidden_channels=64,
    out_channels=dataset_arxiv.num_classes,
    num_heads=8,
    dropout=0.6
).to(device)


optimizer_no_graph = torch.optim.Adam(model_no_graph.parameters(), lr=0.005, weight_decay=5e-4)
scheduler_no_graph = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer_no_graph, mode='min', factor=0.7, patience=10, verbose=True
)

def train_no_graph():
    ## TODO 12: training code for the model without graph structure, should be the same as the original training code (2 points)

    # use the same model but with data without graph structure
    model_no_graph.train()
    optimizer_no_graph.zero_grad()

    # Forward pass with data without the graph structure
    out = model_no_graph(data_arxiv_no_graph)
    
    # Compute loss
    loss = F.cross_entropy(out[train_idx_arxiv], data_arxiv_no_graph.y[train_idx_arxiv].squeeze())
    
    # Backward pass and optimization
    loss.backward()
    optimizer_no_graph.step()

    ## TODO 12: training code for the model without graph structure, should be the same as the original training code
    return loss.item()

@torch.no_grad()
def evaluate_no_graph():
    ## TODO 13: evaluation code for the model without graph structure, should be the same as the original evaluation code (2 points)

    model_no_graph.eval()
    
    # Forward pass with data without the graph structure
    out = model_no_graph(data_arxiv_no_graph)
    y_pred = out.argmax(dim=1).unsqueeze(1) 
    
    # Calculate accuracy for train, valid, and test sets
    train_acc = evaluator_arxiv.eval({'y_true': data_arxiv_no_graph.y[train_idx_arxiv], 'y_pred': y_pred[train_idx_arxiv]})['acc']
    valid_acc = evaluator_arxiv.eval({'y_true': data_arxiv_no_graph.y[valid_idx_arxiv], 'y_pred': y_pred[valid_idx_arxiv]})['acc']
    test_acc  = evaluator_arxiv.eval({'y_true': data_arxiv_no_graph.y[test_idx_arxiv], 'y_pred': y_pred[test_idx_arxiv]})['acc']

    ## TODO 13: evaluation code for the model without graph structure, should be the same as the original evaluation code
    return train_acc, valid_acc, test_acc


## You can change these hyperparameters to see if you can get better results, but the default hyperparameters should work. And also make sure the three sets for hyperparameters are the same.
num_epochs_no_graph = 30
best_valid_acc_arxiv_no_graph = 0
patience_arxiv_no_graph = 30
trigger_times_arxiv_no_graph = 0
best_model_state_arxiv_no_graph = None
# start training and evaluation
for epoch in range(1, num_epochs_no_graph+1):
    loss = train_no_graph()
    scheduler_no_graph.step(loss)
    train_acc, valid_acc, test_acc = evaluate_no_graph()
    ## TODO 14: early stopping, in an if-else block (2 points)

    if valid_acc > best_valid_acc_arxiv_no_graph:
        best_valid_acc_arxiv_no_graph = valid_acc
        best_model_state_arxiv_no_graph = model_no_graph.state_dict()
        trigger_times_arxiv_no_graph = 0  # Reset trigger count when improvement is observed
    else:
        trigger_times_arxiv_no_graph += 1  # Increment trigger count when no improvement
        
    ## TODO 14: early stopping, in an if-else block

    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, Valid: {valid_acc:.4f}, Test: {test_acc:.4f}')

    if trigger_times_arxiv_no_graph >= patience_arxiv_no_graph:
        print("Early stopping triggered!")
        break
# Load the best model state
model_no_graph.load_state_dict(best_model_state_arxiv_no_graph)
final_train, final_valid, final_test = evaluate_no_graph()
print(f"[ogbn-arxiv_no_graph] Best validation accuracy: {final_valid:.4f}, corresponding test accuracy: {final_test:.4f}")


Epoch: 001, Loss: 7.0533, Train: 0.1792, Valid: 0.0766, Test: 0.0587
Epoch: 002, Loss: 6.5416, Train: 0.1429, Valid: 0.2423, Test: 0.2257
Epoch: 003, Loss: 5.9118, Train: 0.2650, Valid: 0.3299, Test: 0.3116
Epoch: 004, Loss: 5.6113, Train: 0.3427, Valid: 0.3552, Test: 0.3180
Epoch: 005, Loss: 5.2832, Train: 0.3241, Valid: 0.3148, Test: 0.2801
Epoch: 006, Loss: 5.0222, Train: 0.2780, Valid: 0.2545, Test: 0.2253
Epoch: 007, Loss: 4.8658, Train: 0.2747, Valid: 0.2611, Test: 0.2304
Epoch: 008, Loss: 4.7486, Train: 0.3013, Valid: 0.3079, Test: 0.2811
Epoch: 009, Loss: 4.5691, Train: 0.3454, Valid: 0.3669, Test: 0.3491
Epoch: 010, Loss: 4.4079, Train: 0.3682, Valid: 0.3999, Test: 0.3833
Epoch: 011, Loss: 4.3124, Train: 0.3751, Valid: 0.4112, Test: 0.3891
Epoch: 012, Loss: 4.2166, Train: 0.4078, Valid: 0.4349, Test: 0.4084
Epoch: 013, Loss: 4.0907, Train: 0.4302, Valid: 0.4444, Test: 0.4163
Epoch: 014, Loss: 3.9909, Train: 0.4111, Valid: 0.4213, Test: 0.3970
Epoch: 015, Loss: 3.9094, Train: 0

* What's the best validation accuracy you can get? (2 points) What's the corresponding test accuracy? (2 points) Please report the results in this markdown cell.
<br><br>**ANS:**
Best Validation Accuracy: 0.4428
Corresponding Test Accuracy: 0.4226

* Let's see if we further remove the edge structure and only use the node features, how well the model can perform.

In [ ]:
import copy

# an empty edge_index, even no self-loops
dummy_edge_index = torch.empty((2, 0), dtype=torch.long, device=data_arxiv.x.device)

# duplicate the original data and replace the edge_index with the dummy one
data_arxiv_no_edge = copy.deepcopy(data_arxiv)
data_arxiv_no_edge.edge_index = dummy_edge_index

# define a new model for the data without graph structure, with the same hyperparameters
model_no_edge = GAT(
    in_channels=dataset_arxiv.num_features,
    hidden_channels=64,
    out_channels=dataset_arxiv.num_classes,
    num_heads=8,
    dropout=0.6
).to(device)

optimizer_no_edge = torch.optim.Adam(model_no_edge.parameters(), lr=0.005, weight_decay=5e-4)
scheduler_no_edge = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer_no_edge, mode='min', factor=0.7, patience=10, verbose=True
)

def train_no_edge():
    ## TODO 15: training code for the model without edge structure, should be the same as the original training code (2 points)

    # use the same model but with data without graph structure
    model_no_edge.train()
    optimizer_no_edge.zero_grad()

    # Forward pass with data without edge structure
    out = model_no_edge(data_arxiv_no_edge)
    
    # Compute the loss
    loss = F.cross_entropy(out[train_idx_arxiv], data_arxiv_no_edge.y[train_idx_arxiv].squeeze())
    
    # Backward pass and optimization
    loss.backward()
    optimizer_no_edge.step()

    ## TODO 15: training code for the model without edge structure, should be the same as the original training code
    return loss.item()

@torch.no_grad()
def evaluate_no_edge():
    ## TODO 16: evaluation code for the model without edge structure, should be the same as the original evaluation code (2 points)

    model_no_edge.eval()

    # Forward pass with data without edge structure
    out = model_no_edge(data_arxiv_no_edge)
    y_pred = out.argmax(dim=1).unsqueeze(1)  # Reshape y_pred to match the shape of y_true

    # Calculate accuracy for train, valid, and test sets
    train_acc = evaluator_arxiv.eval({'y_true': data_arxiv_no_edge.y[train_idx_arxiv], 'y_pred': y_pred[train_idx_arxiv]})['acc']
    valid_acc = evaluator_arxiv.eval({'y_true': data_arxiv_no_edge.y[valid_idx_arxiv], 'y_pred': y_pred[valid_idx_arxiv]})['acc']
    test_acc  = evaluator_arxiv.eval({'y_true': data_arxiv_no_edge.y[test_idx_arxiv], 'y_pred': y_pred[test_idx_arxiv]})['acc']

    ## TODO 16: evaluation code for the model without edge structure, should be the same as the original evaluation code
    return train_acc, valid_acc, test_acc


## You can change these hyperparameters to see if you can get better results, but the default hyperparameters should work. And also make sure the three sets for hyperparameters are the same.
num_epochs_no_edge = 30
best_valid_acc_no_edge = 0
patience_no_edge = 30
trigger_times_no_edge = 0
best_model_state_no_edge = None

# start training and evaluation
for epoch in range(1, num_epochs_no_edge+1):
    loss = train_no_edge()
    scheduler_no_edge.step(loss)
    train_acc, valid_acc, test_acc = evaluate_no_edge()
    ## TODO 17: early stopping, in an if-else block (2 points)

    if valid_acc > best_valid_acc_no_edge:
        best_valid_acc_no_edge = valid_acc
        best_model_state_no_edge = model_no_edge.state_dict()
        trigger_times_no_edge = 0  # Reset trigger count when improvement is observed
    else:
        trigger_times_no_edge += 1  # Increment trigger count when no improvement

    ## TODO 17: early stopping, in an if-else block

    print(f"Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, Valid: {valid_acc:.4f}, Test: {test_acc:.4f}")

    if trigger_times_no_edge >= patience_no_edge:
        print("Early stopping triggered!")
        break

# load the best model state
model_no_edge.load_state_dict(best_model_state_no_edge)
final_train, final_valid, final_test = evaluate_no_edge()
print(f"[ogbn-arxiv_no_edge] Best validation accuracy: {final_valid:.4f}, corresponding test accuracy: {final_test:.4f}")

Epoch: 001, Loss: 6.9605, Train: 0.1791, Valid: 0.0763, Test: 0.0586
Epoch: 002, Loss: 6.5561, Train: 0.1176, Valid: 0.2347, Test: 0.2204
Epoch: 003, Loss: 6.1048, Train: 0.1989, Valid: 0.2981, Test: 0.3183
Epoch: 004, Loss: 5.7291, Train: 0.2972, Valid: 0.2876, Test: 0.2595
Epoch: 005, Loss: 5.3743, Train: 0.3102, Valid: 0.2910, Test: 0.2552
Epoch: 006, Loss: 5.0659, Train: 0.2447, Valid: 0.2001, Test: 0.1749
Epoch: 007, Loss: 4.9449, Train: 0.2503, Valid: 0.2198, Test: 0.1909
Epoch: 008, Loss: 4.8078, Train: 0.2863, Valid: 0.2869, Test: 0.2582
Epoch: 009, Loss: 4.6061, Train: 0.3318, Valid: 0.3591, Test: 0.3436
Epoch: 010, Loss: 4.4422, Train: 0.3622, Valid: 0.4134, Test: 0.4135
Epoch: 011, Loss: 4.3661, Train: 0.3767, Valid: 0.4285, Test: 0.4242
Epoch: 012, Loss: 4.2621, Train: 0.3887, Valid: 0.4230, Test: 0.3997
Epoch: 013, Loss: 4.1627, Train: 0.4141, Valid: 0.4259, Test: 0.3943
Epoch: 014, Loss: 4.0409, Train: 0.4204, Valid: 0.4268, Test: 0.3930
Epoch: 015, Loss: 3.9257, Train: 0

* What's the best validation accuracy you can get? What's the corresponding test accuracy? Please report the results in this markdown cell (2 points).
<br><br>**ANS:**
Best Validation Accuracy: 0.4354
Corresponding Test Accuracy: 0.4122<br><br>
* What's your observation of the results (all the three situations)? Please write down your observation in this markdown cell (2 points).
<br><br>**ANS:**<br>

**Observation of the Results:**

**Full Graph Structure (GAT model):**

The best performance is achieved when the model utilizes both node features and graph edges. The GAT model captures the relationships between nodes effectively, leading to the highest validation and test accuracy.

**No Graph Structure (Only Node Features):**

When the graph structure is removed and the model only has access to node features, performance declines significantly. The model loses the ability to capture inter-node relationships, resulting in a noticeable drop in accuracy on both validation and test sets.

**No Edge Structure (Self-Loops Only):**

When the edge structure is further removed, leaving only self-loops (i.e., each node is connected only to itself), the performance drops even further. The minimal structure provided by self-loops doesn't offer enough information for the model to make accurate predictions, causing the test accuracy to be even lower than the model using only node features.

**Summary:**

The graph structure (edges) plays a crucial role in node classification tasks, as it enables the model to learn relationships between nodes. As we progressively remove the graph structure (first removing all edges, then using only self-loops), the model's performance declines. This highlights how important the topology of the graph is for capturing node dependencies, which is key for accurate predictions in graph-based tasks.



## Graph Convolutional Network (GCN) Implementation for Graph Classification

**Objective:** Implement a GCN from scratch to perform graph classification using an OGB dataset. Develop neural components, including the forward pass, as well as training and testing routines. There are 11 `todo`s and 7 questions for GCN.

## 1. Introduction

Provide an overview of Graph Neural Networks (GNNs) and the significance of GCNs in processing graph-structured data. Discuss the relevance of graph classification tasks and the role of the OGB datasets in benchmarking.

## 2. Dataset Exploration

### 2.1. Importing Libraries



In [5]:
import torch
import torch.nn.functional as F
from torch_geometric.data import DataLoader
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
import torch_geometric

### 2.2. Loading Dataset

In [32]:
# Import necessary modules
from ogb.graphproppred import PygGraphPropPredDataset  # OGB dataset loader for graph property prediction
from torch_geometric.data import DataLoader  # PyG DataLoader for handling batches of graphs

# Load the OGB dataset for molecular property prediction.
# 'ogbg-molhiv' is a graph-level dataset where each graph represents a molecular structure,
# and the task is binary classification to predict whether the molecule inhibits HIV replication.
dataset = PygGraphPropPredDataset(name='ogbg-molhiv')

# Split the dataset into training, validation, and test sets.
# The dataset provides predefined splits to ensure consistency in evaluation.
split_idx = dataset.get_idx_split()
train_dataset = dataset[split_idx['train']]  # Training set
valid_dataset = dataset[split_idx['valid']]  # Validation set
test_dataset = dataset[split_idx['test']]    # Test set

# Create DataLoaders for batch processing.
# The DataLoader enables efficient loading of graphs in batches for training and evaluation.
# - batch_size: Number of graphs per batch.
# - shuffle: Whether to shuffle the dataset at each epoch (only done for training).
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)   # Shuffle for training
valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False)  # No shuffle for validation
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)    # No shuffle for testing

c:\Users\monat\AppData\Local\Programs\Python\Python311\Lib\site-packages\ogb\graphproppred\dataset_pyg.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.s

In [33]:
# Display dataset information
print(f'Dataset name: {dataset.name}')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of classes: {dataset.num_tasks}')
print(f'Number of node features: {dataset.num_node_features}')

Dataset name: ogbg-molhiv
Number of graphs: 41127
Number of classes: 1
Number of node features: 9


## 3. Model Implementation
### 3.1. GCN Convolution Layer Implementation

In [34]:
import torch.nn as nn
from torch_geometric.nn import MessagePassing  # Base class for defining message-passing layers
from torch_geometric.utils import add_self_loops, degree  # Utilities for graph processing

class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        """
        Custom implementation of a Graph Convolutional Network (GCN) layer.

        Args:
            in_channels (int): Number of input node features.
            out_channels (int): Number of output node features.

        The layer follows the formulation:
            H' = σ(D^(-1/2) A D^(-1/2) H W)
        where:
            - A is the adjacency matrix with self-loops.
            - D is the degree matrix.
            - H is the input node feature matrix.
            - W is the trainable weight matrix.
            - σ is a non-linearity (like ReLU).
        """
        super(GCNConv, self).__init__(aggr='add')  # "Add" aggregation means summing messages from neighbors.

        # TODO 1: Define a linear transformation layer for node features (2 points)

        self.linear = nn.Linear(in_channels, out_channels)

        # TODO 1: Define a linear transformation layer for node features

    def forward(self, x, edge_index):
        """
        Forward pass of the GCN layer.

        Args:
            x (Tensor): Node feature matrix of shape [num_nodes, in_channels].
            edge_index (Tensor): Graph connectivity in COO format, shape [2, num_edges].

        Returns:
            Tensor: Updated node features of shape [num_nodes, out_channels].
        """

        # TODO 2: Add self-loops to the adjacency matrix (2 points)

        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # TODO 2: Add self-loops to the adjacency matrix

        # TODO 3: Compute node degrees (2 points)

        row, col = edge_index  # Extract row (source nodes) and col (destination nodes)
        deg = degree(row, x.size(0), dtype=x.dtype)  # Compute degree of each node
        deg_inv_sqrt = deg.pow(-0.5)  # Compute D^(-1/2)

        # TODO 3: Compute node degrees 

        # Prevent division by zero for isolated nodes
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0  

        # TODO 4: Compute normalized adjacency matrix (2 points)

        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]  # Normalize using degrees

        # TODO 4: Compute normalized adjacency matrix 

        # TODO 5: Perform message passing using PyTorch Geometric's propagate function (2 points)

        out = self.propagate(edge_index, x=x, norm=norm)

        # TODO 5: Perform message passing using PyTorch Geometric's propagate function 

        # TODO 6: Apply the linear transformation after aggregation (2 points)

        out = self.linear(out)

        # TODO 6: Apply the linear transformation after aggregation 

        return out

    def message(self, x_j, norm):
        """
        Message function: Defines how information is aggregated from neighboring nodes.

        Args:
            x_j (Tensor): Features of neighboring nodes.
            norm (Tensor): Normalization coefficients computed from node degrees.

        Returns:
            Tensor: Normalized node features.
        """
        # TODO 7: Apply normalization to the node features (2 points)

        return norm.view(-1, 1) * x_j 
    
        # TODO 7: Apply normalization to the node features

## TODO:
### Why is it important to add self-loops in the graph convolution process, and how does the normalization strategy here help stabilize training in graph neural networks? (2 points)

**ANS:**

**Importance of Adding Self-Loops:**

Self-loops are crucial in graph convolutions as they allow each node to retain its own information during the aggregation process. Without self-loops, nodes could lose their identity across layers, leading to ineffective learning.

**Normalization Strategy for Stabilizing Training:**

The normalization strategy, using the inverse square root of node degrees, helps stabilize training by preventing nodes with high degrees from dominating the message aggregation. This ensures balanced information flow, avoids exploding gradients, and promotes stable learning in deep networks.

### 3.2. Implement GCN

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import global_mean_pool

class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes, dropout_rate):
        """
        Graph Convolutional Network (GCN) for graph classification.

        Args:
            num_node_features (int): Number of input node features.
            hidden_channels (int): Number of hidden units in the GCN layers.
            num_classes (int): Number of output classes (for classification tasks).
            dropout_rate (float): Dropout probability for regularization.

        This model consists of:
        - Two GCNConv layers to extract graph structure-aware features.
        - ReLU activation and dropout for regularization.
        - A global mean pooling layer to aggregate node features into graph-level representations.
        - A final linear layer for classification.
        """
        super(GCN, self).__init__()

        self.dropout_rate = dropout_rate  # Store dropout rate

        # TODO 8: Complete the network layer definitions (2 points)

        # First graph convolution layer: transforms node features to hidden dimension
        self.gcn1 = GCNConv(num_node_features, hidden_channels)

        # Second graph convolution layer: refines node embeddings
        self.gcn2 = GCNConv(hidden_channels, hidden_channels)

        # Final linear layer to produce class predictions from the pooled graph representation
        self.fc = nn.Linear(hidden_channels, num_classes)

        # TODO 8: Complete the network layer definitions (2 points)

    def forward(self, x, edge_index, batch):
        """
        Forward pass through the GCN.

        Args:
            x (Tensor): Node feature matrix of shape [num_nodes, num_node_features].
            edge_index (Tensor): Graph connectivity in COO format [2, num_edges].
            batch (Tensor): Batch index for each node, used for global pooling.

        Returns:
            Tensor: Output class logits of shape [num_graphs, num_classes].
        """

        # TODO 9: Complete the implementation of the forward function (2 points)

        # First GCN layer: apply convolution, activation, and dropout
        x = self.gcn1(x, edge_index)
        x = F.relu(x)  # ReLU activation layer
        x = F.dropout(x, p=self.dropout_rate, training=self.training)  # Dropout layer for regularization

        # Second GCN layer
        x = self.gcn2(x, edge_index)
        x = F.relu(x)  # ReLU activation layer
        x = F.dropout(x, p=self.dropout_rate, training=self.training)  # Dropout layer for regularization

        # Global mean pooling to aggregate node features into a graph representation
        x = global_mean_pool(x, batch)  # Output shape: [num_graphs, hidden_channels]

        # Final linear layer for classification
        x = self.fc(x)  # Output shape: [num_graphs, num_classes]

        # TODO 9: Complete the implementation of the forward function (2 points)
        return x

## 4. Model Training
### 4.1. Implement Training and Evalutation Function

In [36]:
def train(model, loader, device):
    """
    Training function for the GCN model.

    Args:
        model (torch.nn.Module): The GCN model.
        loader (DataLoader): DataLoader for the training set.
        criterion (torch.nn.Module): Loss function.
        optimizer (torch.optim.Optimizer): Optimizer for training.
        device (torch.device): Device to run computations on (CPU or GPU).

    Returns:
        float: Average training loss per graph.
    """

    model.train()
    total_loss = 0
    # Iterate over batches in the DataLoader
    for data in loader:
        data = data.to(device)

        # TODO 10: Complete the training function (2 points)

        #  Zero the gradients of the Adam optimizer from `torch.optim`
        optimizer.zero_grad()

        # Perform a forward pass through the model
        out = model(data.x, data.edge_index, data.batch)

        # Compute the loss use `BCEWithLogitsLoss()``

        # This loss is suitable for binary classification tasks (e.g., graph classification)
        loss = F.binary_cross_entropy_with_logits(out, data.y.float())

        #  Perform backpropagation
        loss.backward()

        # Update model parameters
        optimizer.step()

        # TODO 10: Complete the training function

        # Accumulate loss
        total_loss += loss.item() * data.num_graphs

    return total_loss / len(loader.dataset)

## TODO:
### How does the choice of the BCEWithLogitsLoss and the Adam optimizer influence the training dynamics in this setting? (2 points)

**ANS:**

1. **BCEWithLogitsLoss**:
   - **BCEWithLogitsLoss** is a loss function specifically designed for binary classification tasks, where the outputs are logits (raw, unnormalized scores) rather than probabilities. It combines a **sigmoid activation** with **binary cross-entropy loss**, which makes it numerically stable by avoiding potential issues with small values from separate sigmoid and loss operations.
   - **Effect on Training**: Since the GCN model produces logits (not probabilities), BCEWithLogitsLoss directly compares the raw logits with the target labels. This is ideal for graph-based binary classification tasks, as it allows the model to learn and update its parameters efficiently while avoiding computational issues related to individual sigmoid and cross-entropy steps.

2. **Adam Optimizer**:
   - **Adam** (short for Adaptive Moment Estimation) is an adaptive learning rate optimization algorithm that computes individual learning rates for different parameters based on estimates of first and second moments (mean and uncentered variance). It combines the benefits of **Momentum** and **RMSprop**, making it well-suited for training deep learning models.
   - **Effect on Training**: Adam adapts the learning rate for each parameter, which helps speed up training and provides robustness against poor initialization. This is particularly useful in graph-based models where the structure of the data can vary significantly. Adam's ability to adapt to the landscape of the loss function helps the model converge faster and more reliably, especially when dealing with complex graph structures.


### What are potential alternative choices and their pros/cons for graph-based binary classification tasks? (2 points)

**ANS:**
1. **Alternative Loss Functions**:
   
   - **CrossEntropyLoss (for multi-class classification)**:
     - **Pros**: Useful if the binary classification task is extended to multi-class problems, as it can handle multi-class classification directly.
     - **Cons**: If the task is strictly binary, BCEWithLogitsLoss would be more appropriate since CrossEntropyLoss requires one-hot encoded labels and can be less efficient for binary tasks.
   
   - **Hinge Loss (SVM-based)**:
     - **Pros**: Effective for margin-based classification tasks where you care about not just correct classification but also the confidence in classification. It can lead to better generalization.
     - **Cons**: Not as commonly used in graph neural networks, especially for tasks like graph classification, where BCEWithLogitsLoss typically provides a smoother and more efficient training process.

2. **Alternative Optimizers**:
   
   - **Stochastic Gradient Descent (SGD)**:
     - **Pros**: One of the simplest and most widely used optimizers, suitable for large-scale datasets and relatively simple models. It is a good option if computational resources are limited.
     - **Cons**: Does not adapt learning rates, making it slower to converge in deep learning tasks. Requires careful tuning of the learning rate and momentum parameters, which can be cumbersome.

   - **RMSprop**:
     - **Pros**: Like Adam, it adapts the learning rate based on recent gradient information, but it does so by maintaining a moving average of squared gradients. Works well with non-stationary objectives.
     - **Cons**: While it adapts learning rates like Adam, it can be less robust for certain tasks and can require tuning of additional hyperparameters like the decay term.

### **Summary**:
- **BCEWithLogitsLoss** is ideal for binary classification tasks where the outputs are logits, and **Adam** is well-suited for the dynamic and complex landscape of graph data, providing fast convergence and reliable training.
- **Alternative loss functions** like **CrossEntropyLoss** or **Hinge Loss** may be useful for different types of classification tasks, but they are generally less efficient or appropriate for binary graph classification tasks.
- **Alternative optimizers** like **SGD** or **RMSprop** could work but may require more hyperparameter tuning and could converge more slowly compared to Adam in graph-based models.

In [37]:
def evaluate(model, loader, evaluator, device):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data.x, data.edge_index, data.batch)
            y_true.append(data.y.view(-1, 1))
            y_pred.append(out)
    y_true = torch.cat(y_true, dim=0)
    y_pred = torch.cat(y_pred, dim=0)
    return evaluator.eval({'y_true': y_true, 'y_pred': y_pred})

## TODO:
### What potential pitfalls can arise during the evaluation phase of graph neural networks? (2 points)

**ANS:**

1. **Data Leakage**:
   - One common pitfall in evaluation is **data leakage**, where information from the validation or test set leaks into the model during training, leading to overly optimistic evaluation results. In graph-based models, leakage can occur if the graph structures or node features are inadvertently shared between training and test data (e.g., improper splitting of data, shared node identifiers across splits, etc.).
   - **Solution**: Always ensure proper data splits and that the validation/test graphs are completely independent from training graphs. In graph classification tasks, using **graph-level splits** ensures no leakage.

2. **Overestimating Model Performance**:
   - **Evaluation on the same dataset used for training** or performing **evaluation without proper validation** can result in overly optimistic performance metrics. For example, running evaluations on the training set or testing on graphs that have similar features can lead to inflated metrics.
   - **Solution**: Evaluate on a **separate validation/test set** that has not been seen by the model during training. Also, ensure the evaluation follows proper cross-validation procedures where applicable.


### How might one detect issues like overfitting or underfitting using evaluation metrics such as ROC-AUC? (2 points)

**ANS:**

1. **Overfitting**:
   - Overfitting occurs when a model performs well on the training data but poorly on the validation/test set. It can be detected by monitoring the **ROC-AUC** (Receiver Operating Characteristic - Area Under Curve) or any other performance metric during training and evaluation.
   - **Detection**: If the ROC-AUC on the training set is **significantly higher** than on the validation/test set, this is a clear indication of overfitting. The model has learned the specifics of the training data, but it fails to generalize to unseen data.
   - **Solution**: To mitigate overfitting, use techniques like **regularization**, **dropout**, **early stopping**, or **cross-validation** to ensure the model generalizes well.

2. **Underfitting**:
   - Underfitting occurs when a model fails to learn the underlying patterns in the data, typically due to an overly simplistic model or inadequate training. It can be detected if both the training and validation/test ROC-AUC scores are **significantly low**.
   - **Detection**: If the ROC-AUC score is low on both the training and test sets, it indicates that the model has not learned the relevant features, possibly due to insufficient complexity (e.g., too few layers or features) or inadequate training.
   - **Solution**: To address underfitting, increase the model complexity (e.g., adding more layers, increasing hidden units), provide better features, or train for more epochs to allow the model to learn more effectively.

### **Summary**:
- **Pitfalls**: Issues like data leakage or overestimating performance can arise if proper data splits and independent evaluation sets are not used.
- **Detection of Overfitting/Underfitting**: Using **ROC-AUC**, overfitting is detected when performance on the training set is much better than on the validation/test set, and underfitting is detected when performance is low on both.

### 4.2. Training Loop (Hyper-parameter Tuning to reach `Validation AUC` $\ge$ 0.7 ) 

1 pt: `Validation AUC` $\ge$ 0.69


1.5 pts: `Validation AUC` $\ge$ 0.71


2 pts: `Validation AUC` $\ge$ 0.73

In [38]:
# TODO 11: Tune your hyper-parameter to achieve high performance (2 points)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(
    num_node_features=dataset.num_node_features, 
    hidden_channels=128,  # Increased hidden units
    num_classes=dataset.num_tasks, 
    dropout_rate=0.3
).to(device)
evaluator = Evaluator(name='ogbg-molhiv')

# Initialize the Adam optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005, betas=(0.85, 0.999), weight_decay=1e-5)

# Learning Rate Scheduler (Cosine Annealing for smooth decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

# TODO 11: Tune your hyper-parameter to achieve high performance

### 4.2. Training Loop (Hyper-parameter Tuning to reach `Validation AUC` $\ge$ 0.7 ) 

1 pt: `Validation AUC` $\ge$ 0.69


1.5 pts: `Validation AUC` $\ge$ 0.71


2 pts: `Validation AUC` $\ge$ 0.73

In [39]:
from tqdm import tqdm
num_epochs = 150
best_valid_auc = 0
patience = 20      # Early stopping patience to avoid overfitting
trigger_times = 0  # Counter for early stopping

for epoch in tqdm(range(1, num_epochs + 1)):
    # Training
    train_loss = train(model, train_loader, device)

    # Evaluation
    train_result = evaluate(model, train_loader, evaluator, device)
    valid_result = evaluate(model, valid_loader, evaluator, device)

    train_auc = train_result['rocauc']
    valid_auc = valid_result['rocauc']

    # Print metrics
    print(f'Epoch: {epoch:03d}, '
          f'Train Loss: {train_loss:.4f}, '
          f'Train AUC: {train_auc:.4f}, '
          f'Validation AUC: {valid_auc:.4f}')

    # Save the best model
    if valid_auc > best_valid_auc:
        best_valid_auc = valid_auc
        # Save the model with the best validation AUC
        torch.save(model.state_dict(), 'best_model.pth')
        print(f'Saved best model with Validation AUC: {best_valid_auc:.4f}')
        trigger_times = 0  # Reset early stopping trigger
    else:
        trigger_times += 1  # Increment early stopping trigger
    
    # Early stopping if validation AUC does not improve for `patience` epochs
    if trigger_times >= patience:
        print(f"Early stopping at epoch {epoch} due to no improvement in Validation AUC.")
        break

  1%|          | 1/150 [00:13<33:54, 13.65s/it]

Epoch: 001, Train Loss: 0.1781, Train AUC: 0.4759, Validation AUC: 0.5613
Saved best model with Validation AUC: 0.5613


  1%|▏         | 2/150 [00:21<25:40, 10.41s/it]

Epoch: 002, Train Loss: 0.1600, Train AUC: 0.5736, Validation AUC: 0.6429
Saved best model with Validation AUC: 0.6429


  2%|▏         | 3/150 [00:29<22:40,  9.25s/it]

...

Epoch: 142, Train Loss: 0.1428, Train AUC: 0.7495, Validation AUC: 0.7251


 95%|█████████▌| 143/150 [17:37<00:52,  7.55s/it]

Epoch: 143, Train Loss: 0.1426, Train AUC: 0.7476, Validation AUC: 0.7363


 95%|█████████▌| 143/150 [17:44<00:52,  7.44s/it]

Epoch: 144, Train Loss: 0.1439, Train AUC: 0.7516, Validation AUC: 0.7373
Early stopping at epoch 144 due to no improvement in Validation AUC.


In [40]:
# Load the best model for final evaluation
model.load_state_dict(torch.load('best_model.pth'))

# Final evaluation on the test set
test_result = evaluate(model, test_loader, evaluator, device)
test_auc = test_result['rocauc']
print(f'Final Test AUC: {test_auc:.4f}')

C:\Users\monat\AppData\Local\Temp\ipykernel_7432\675879185.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Final Test AUC: 0.6865


### OPEN ENDED QUESTION:
#### Reflect on the entire implementation of the GCN model for molecular property prediction.
#### - What potential improvements or modifications could you suggest to enhance the model's performance? Consider aspects such as network architecture, hyperparameter tuning, data preprocessing, and evaluation strategies. (2 points)

#### **ANS: Potential Improvements & Modifications for Enhancing Performance**
Several aspects of the current GCN model can be improved to achieve better performance:

1. **Network Architecture Modifications:**
   - Introduce **Graph Attention Networks (GAT)** instead of GCN to allow adaptive weighting of neighboring nodes.
   - Use **Graph Isomorphism Network (GIN)**, which has been shown to be more powerful in capturing molecular graph structures.
   - Add **residual connections** between layers to prevent vanishing gradients and allow deeper architectures.

2. **Hyperparameter Tuning:**
   - Perform a **grid search** or **Bayesian optimization** over hidden dimensions, learning rate, dropout, and weight decay.
   - Try **learning rate warm-up** before applying decay to stabilize training.
   - Experiment with **larger batch sizes** (e.g., 512) to achieve smoother gradient updates.

3. **Data Preprocessing & Augmentation:**
   - Apply **feature engineering** on molecular graphs (e.g., adding edge attributes for bond types).
   - Consider **self-supervised pretraining** using contrastive learning techniques like **GraphCL** to improve node representations before fine-tuning.

4. **Evaluation Strategies:**
   - Use **stratified k-fold cross-validation** to ensure robustness in model evaluation.
   - Implement **class balancing techniques** to handle label imbalance in molecular property datasets.
   - Monitor **precision-recall curves** in addition to ROC-AUC for better insight into class performance.

---
#### - How might you leverage additional graph-specific techniques or modern architectures to push the boundaries of performance on this task? (2 points)

#### **ANS: Leveraging Additional Graph-Specific Techniques & Modern Architectures**
Beyond traditional GCNs, we can incorporate **more advanced graph neural network techniques**:

1. **Graph Transformers:**
   - Use **Graphormer** or **SAN (Structure-Aware Transformer)** to capture long-range dependencies in molecular graphs.
   - These models **outperform standard GNNs** on certain molecular property prediction tasks.

2. **Contrastive Learning for Graph Representations:**
   - Implement **Graph Contrastive Learning (GraphCL, InfoGraph)** to pretrain the model on unlabeled molecular graphs.
   - This can significantly improve feature representations and generalization.

3. **Multi-View Graph Learning:**
   - Use **multi-scale GCNs** that extract information at **different neighborhood levels**.
   - Combine **node-level and subgraph-level embeddings** for richer representations.

4. **Combining GNNs with Domain Knowledge:**
   - Incorporate **physically meaningful descriptors** (e.g., quantum chemical properties) as input features.
   - Hybrid models that integrate **GNNs with traditional cheminformatics features** (e.g., molecular fingerprints) can enhance prediction accuracy.

5. **Meta-Learning for Adaptive GNN Training:**
   - Use **meta-learning (MAML, Reptile)** to adapt the model quickly to different molecular tasks.
   - Helps in few-shot learning scenarios where labeled data is limited.


### **Summary**
To push the boundaries of performance, we can:
Adopt **advanced GNN architectures** (GAT, GIN, Graph Transformers).  
Use **contrastive learning & self-supervised pretraining**.  
Explore **multi-scale graph learning** & integrate domain knowledge.  
Improve **evaluation strategies & hyperparameter tuning**.

By combining these approaches, we can significantly enhance the model's effectiveness for molecular property prediction